# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.  
  - **Title**: The title of the movie.                     
  - **Year**: The year the movie was released.           
  - **Rating**: The IMDb rating of the movie.            
  - **Runtime (min)**: The duration of the movie in minutes.   
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.                  
  - **Director**: The director of the movie.                   
  - **Stars**: The main stars of the movie.                    
  - **Votes**: The number of votes the movie received.          
  - **Gross** : The gross earnings of the movie in millions of USD.    

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [3]:
def scrape_imdb(title_type, user_rating, start_date, end_date):
    import pandas as pd
    from bs4 import BeautifulSoup
    import requests
    import re
    
    
    url = f"https://www.imdb.com/search/title/?title_type={title_type}&release_date={start_date},{end_date}&user_rating={user_rating}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    
    title_element = soup.find_all("h3", class_ ="ipc-title__text")
    titles_lists = [i.get_text().split(".") for i in title_element]

    # 1
    nr = [i[0] for i in titles_lists[:25]]
    #print(len(nr))
    
    # 2
    titles = [i[1].strip() for i in titles_lists[:25]]
    #print(len(titles))
    
    # 3
    year_element = soup.find_all("span", class_ = "sc-15ac7568-7 cCsint dli-title-metadata-item")
    valid_years = [str(y) for y in range(int(start_date.split("-")[0]), int(end_date.split("-")[0]) + 1)]
    year_lists = [i.get_text() for i in year_element if i.get_text() in valid_years]
    #print(len(year_lists))
    
    # 4
    rating_element = soup.find_all('span', class_="ipc-rating-star--rating")
    rating_lists = [i.get_text() for i in rating_element]
    #print(len(rating_lists))
    
    # 5
    runtime_element = soup.find_all("span", class_ = "sc-15ac7568-7 cCsint dli-title-metadata-item")
    runtime_lists = [i.get_text() for i in runtime_element]
    runtime = runtime_lists[1:74:3]
    #print(len(runtime))
    
    # 6
    description_elements = soup.find_all("div", class_ = 'ipc-html-content-inner-div')
    description = [i.get_text() for i in description_elements]
    #print(len(description))
    
    # 7
    votes_element = soup.find_all("span", class_ = "ipc-rating-star--voteCount")
    votes = [i.get_text() for i in votes_element]
    vote = [i.replace("\xa0(","").replace(")","") for i in votes]
    #print(len(vote))
    

    #Links of Movies
    link_element = soup.find_all("a", class_ = "ipc-lockup-overlay ipc-focusable")
    list_of_links = [i.get("href") for i in link_element[:25]]

    # Empty Lists
    list_of_urls= []
    gross_money_list =[]
    list_of_directors = []
    list_of_actors = []
    list_of_writers = []
    list_of_genres = []

    # Rotating over every link
    for u in list_of_links:
        url = f"https://www.imdb.com{u}"

        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        # 8 Gross
        gross_element = soup.find_all("span",class_ = "ipc-metadata-list-item__list-content-item ipc-btn--not-interactable")
        gross_money_list.append(gross_element[3].get_text())

        # Director + Writers + Actors
        list_of_directors_in_a_movie = []
        list_of_writers_in_a_movie = []
        list_of_actors_in_a_movie  = []
        people_element = soup.find_all("a", class_ = "ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
        for i in people_element:
            person = i.get('href') 

            # 9 Director
            if re.search(r'_1_\d$', person):
                director = i.get_text()
                if director not in list_of_directors_in_a_movie:
                    list_of_directors_in_a_movie.append(director)

            # 10 Writer
            elif re.search(r'_2_\d$', person):
                writer = i.get_text()
                if writer not in list_of_writers_in_a_movie:
                    list_of_writers_in_a_movie.append(writer)        

            # 11 Actor
            elif re.search(r'_3_\d$', person):
                actor = i.get_text()
                if actor not in list_of_actors_in_a_movie:
                    list_of_actors_in_a_movie.append(actor)

        list_of_directors.append(list_of_directors_in_a_movie)
        list_of_writers.append(list_of_writers_in_a_movie)
        list_of_actors.append(list_of_actors_in_a_movie)

        # 12 Genres
        genre = soup.find("div",class_ = "ipc-chip-list__scroller").get_text()
        parts = re.findall(r'[A-Z][^A-Z]*', genre)
        genres = [p.strip() for p in parts if p.strip()]
        list_of_genres.append(genres)


        list_of_directors_str = [", ".join(d) for d in list_of_directors]
        list_of_writers_str = [", ".join(w) for w in list_of_writers]
        list_of_actors_str = [", ".join(a) for a in list_of_actors]
        list_of_genres_str = [", ".join(g) for g in list_of_genres]
  
    # Formatting the dataframe
    df = pd.DataFrame({"Number": nr, "Title": titles, "Year of Release": year_lists,
                       "Rating": rating_lists, "Voting":vote, "Runtime (min)": runtime,
                       "Description": description, "Director": list_of_directors_str,
                        "Writers": list_of_writers_str,"Stars": list_of_actors_str,
                        "Genres": list_of_genres_str, "Gross": gross_money_list})
    
    return df
#run time

In [ ]:
df_scraped = scrape_imdb("feature", 7.0, "2000-01-01", "2002-12-31")
df_scraped

In [ ]:
#df_scraped.to_csv("df_scraped.csv")